In [15]:
from typing import Any, Dict, List
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import (
    FewShotPromptTemplate,
    PromptTemplate,
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain_openai import ChatOpenAI
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here I know this:
        Capital: Paris
        Language: French
        Food: Croissant, Baguette, Cheese, Wine
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        Here I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza, Pasta, Gelato,
        Currency: Euro
        """,
    },
    {
        "country": "Spain",
        "answer": """
        Here I know this:
        Capital: Madrid
        Language: Spanish
        Food: Paella, Tapas, Churros
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example: Dict[str, str]) -> Any:
        self.examples.append(example)

    def select_examples(self, input_variables) -> List[dict]:
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template(
    "Human: What do you know about {country}?\nAI: {answer}"
)
example_selector = RandomExampleSelector(examples=examples)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

TypeError: string indices must be integers, not 'str'